# Phase 2 Final Project

* Student name: Caroline Surratt
* Student pace: Self-Paced
* Scheduled project review date/time: Wednesday, August 23rd at 3:00 PM
* Instructor name: Morgan Jones

# Business Understanding

This analysis was conducted for a company that is interested in creating original video content. However, they lack insight as to what types of films are currently performing best at the box office. This analysis seeks to understand patterns and trends in movie performance in order to recommend what types of films the company should produce to maximize profit. 

For the purpose of this analysis, movie performance will be evaluated using **percent return on investment**. 



\begin{align}
\text{Percent Return} = \frac{\text{total gross revenue – production budget}}{\text{production budget}}*100
\end{align}


While other factors (consumer ratings, critic reviews, etc.) may be meaningful in the context of analyzing film as an art form, this company is not necessarily interested in producing Academy-Award-winning films. Rather, they are concerned with earning a profit and maximizing their investments in the new movie studio.

In order to better understand what types of films are performing best, I will examine the relationships between percent return and movie characteristics, including production budget, genre, and runtime. 

# Data Understanding

Two datasets were used for this analysis.

The first dataset is from The Numbers. This dataset contains information about nearly 6,000 film budgets and revenues from 1915-2020, with most entries falling between the years 2000-2012. This dataset is stored in this repository as a compressed CSV file.

The second dataset is from IMDB. This dataset is a SQLite database with information about over 146,000 movies, including information about each movie's length, when it was released, and its genre. This database also contains information about actors/actresses, directors, and writers, as well as which films they are associated with.

White there are significantly more entries in IMDB, this information will not be useful to the company without additional information about the profitability of the movie. Therefore, only entries in IMBD that have a corresponding entry in The Numbers dataset will be considered.

# Importing Data and Packages

First, I will import the libraries and packages necessary for this analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
from scipy import stats
import seaborn as sns
import statsmodels.api as sm

%matplotlib inline

#turned off chained assignment warning, as I was receiving false positive warnings
pd.options.mode.chained_assignment = None

### The Numbers Dataset

Now, I will import the dataset from The Numbers and preview the DataFrame.

In [ ]:
numbers = pd.read_csv("data/tn.movie_budgets.csv.gz")
numbers

In [ ]:
numbers.info()

This dataset contains information about the budgets and revenues of nearly 6,000 movies, with no missing entries. 

Before much of the data can be utilized, the numbers must be converted to integers instead of objects. For the columns with currency values, I will remove the $ symbol and commas in order to convert these values to a usable type.

The only necessary information from the release_date column is the year, as this is the information that is also contained in the IMDB table. Therefore, I will also adjust the release_date column to only contain the year. This way,  this column can be used to match movies to their corresponding entries across the two datasets.

In [ ]:
# created a list of the columns that are referring to a currency
currency_columns = ['production_budget', 'domestic_gross', 'worldwide_gross']

# used a for loop to remove '$' and ',' symbols from the entries in these columns and convert the remaining
# characters to integers instead of strings
for column in currency_columns:
    numbers[column] = numbers[column].str.replace("$", "")
    numbers[column] = numbers[column].str.replace(",", "")
    numbers[column] = numbers[column].astype(int)

# used a labmda function to select only the year from the release_date column and dropped the column with entire date
numbers['release_year'] = numbers['release_date'].map(lambda x: int(x[-4:]))
numbers.drop('release_date', axis=1, inplace=True)

Now that the columns with numeric values are of a numeric type, we can take a look at the descriptive statistics for this dataset, beginning with the release year.

In [ ]:
numbers['release_year'].describe()

This dataset contains entries about movies that were released as early as 1915. Because the company is interested in films that are currently doing well and film trends have changed significantly in the last century, I will limit this analysis to films that were produced in the 21st century (the year 2000 or later). This will still leave a substantial portion (over 75%) of the dataset for us to work with.

In [ ]:
numbers = numbers.loc[numbers['release_year'] >= 2000]

### IMDB Dataset

In order to better understand the features of these movies and evaluate whethere there are trends and patterns in the data, we will need to incorporate additional information from the IMDB file. In the cell below, I will connect to the IMDB file using SQLite 3. Then, I will merge the IMDB data with the data from The Numbers. Entries that have insufficient information will not be useful to this analysis: therefore, an inner merge will be used.

In [ ]:
# from zipfile import ZipFile
# with ZipFile("data/im.db.zip", 'r') as zObject:
    #z Object.extractall(path="data/im.db")

conn = sqlite3.connect("data/im.db/im.db")

imdb = pd.read_sql("""
SELECT original_title, 
       start_year, 
       runtime_minutes,
       genres
FROM movie_basics;""", conn)

In [ ]:
conn.close()

In [ ]:
movies = numbers.merge(imdb, how='inner', left_on=['movie', 'release_year'], 
                       right_on=['original_title', 'start_year'])

In [ ]:
movies.describe()

Now, we have a set of 1,533 movies produced since 2000 for which we have information about their production budget, revenue, release year, runtime, and genre.

The statistics show that on average, these films cost approximately \\$44,00,000  to produce, and on average, they brought in approximately \\$137,000,000 in worldwide gross revenue.

Looking at total gross revenue will give us some sense of what types of films perform best overall, but it will also be useful to consider the relationship between revenue and production budget. Therefore, I will create a new column called "%_return" that gives the gross revenue as a percentage of the production budget. The formula used can be seen in the cell below. I will also sort the dataframe by this column so that the most profitable films are easily accessible.

In [ ]:
# percent return = (earnings - cost) / cost
movies['%_return'] = (movies['worldwide_gross']-movies['production_budget'])/movies['production_budget']*100
movies.sort_values(by=['%_return'], ascending=False, inplace=True)

In [ ]:
movies['%_return'].describe()

On average, these movies earned a 269% return on their initial investment. That is, a film earned 2.5 times what was spent to produce the film. The film with the highest percent return had a whopping 41,556% return on investment! The lowest films had a -100% return (that is, no money was earned from the film, so the initial investment was lost).

In order to visually examine the relationship between these variables (production budget, total revenue, and percent return), I will create scatterplots below.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8))

movies.plot.scatter(x="production_budget", y="worldwide_gross", ax=axes[0])
axes[0].set_xlabel("Production Budget")
axes[0].set_ylabel("Worldwide Gross Revenue")

movies.plot.scatter(x="production_budget", y="%_return", ax=axes[1])
axes[1].set_xlabel("Production Budget")
axes[1].set_ylabel("Percent Return");

The scatter plot on the left illustrates the relationship between production budget and worldwide gross revenue. Although this relationship is not linear and does not meet the criteria for performing a linear regression, there does appear to be somewhat of a weak relationship between budget and revenue: films with larger budgets tend to have larger revenues. 

The scatter plot to the right illustrates the relationship between production budget and percent return on the budget. Interestingly, the films with the highest percent returns have some of the lowest budgets. These films will be of particular interest to the company, as they demonstrate opportunities to generate impressive returns on investments.

In order to better understand the relationship between films with the highest percent return on investment, I binned the data according to this feature and examined the budget using the function below.

In [ ]:
def calculate_production_budget(n):
    top_n = movies.head(n)
    production_budget = top_n['production_budget'].mean()
    print("The production budget for the ", n, " films with the highest percent return is $", 
          round(production_budget), ".")

In [ ]:
for n in 5, 30, 50, 100, 250, 500:
    calculate_production_budget(n)

Again, there is not a linear relationship between the production budget and the percent return, but average production budgets produced above illustrate the following: The films that earn the most money per dollar (that is, the films that have the highest return on investment) have lower budgets, on average, than other films.

This led me to question: Are the budgets of these highly profitable movies different from the budgets of all other movies? In order to examine this question, I will use a t-test.

**Null hypothesis**: Highly profitable movies have the same production budgets as all other movies.

**Alternative hypothesis**: Highly profitable movies have the different production budgets than other movies.

I will consider highly profitable movies to be the 30 movies with the highest percent return on investment.

In [ ]:
subset = movies.head(30)
subset.describe()

On average, these top 30 movies have a production budget of $5,360,000, which is much lower than the average budget for all movies. 

Is this difference statistially significant?

In [ ]:
sample = subset['production_budget'].to_numpy()

In [ ]:
mu = numbers['production_budget'].mean()

In [ ]:
stats.ttest_1samp(a=sample, popmean=mu)

Using the t-statistic and p-value calculated above, we can **reject the null hypothesis** at the standard alpha level of 0.05: it is extremely unlikely that the production budgets of highly profitable movies are a subset of the productiuon budgets of all movies. 

In [ ]:
sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(10,6))

sns.histplot(movies['production_budget'], kde=True, bins=102, \
             stat='probability', alpha=0.4, ax=ax, color='blue', label = "All Movies")

sns.histplot(subset['production_budget'], kde=True, bins=5, \
             stat='probability', alpha=0.5, ax=ax, color='red', label="Top 30 Movies")

plt.xlim(-10, 100000000)
plt.xticks(ticks=[0, 20000000, 40000000, 60000000, 80000000, 100000000], labels=[0, 20, 40, 60, 80, 100])
plt.xlabel("Production Budget (in millions of dollars)")
plt.title("Distribution of Production Budgets", fontsize=18)
plt.legend();

We cannot assume that the production budget *caused* these films to be more profitable. However, this difference in production budget is a statistically significant feature of highly profitable movies.

**Therefore, I recommend that the company mimic the production budgets of these highly profitable films and limit production budgets to $2 million.**

### Breaking Down Movies by Genre

In [ ]:
all_genres = list(movies['genres'].value_counts().index)

In [ ]:
# https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

def split_and_flatten(list):
    split_list = []
    for entry in list:
        split_list.append(entry.split(','))
    return set([item for sublist in split_list for item in sublist])

genres = split_and_flatten(all_genres)
genres

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

def number_in_genre(genres_list):
    dataframes = []
    for genre in genres_list:
        q = """SELECT COUNT(*) 
               FROM movies 
               WHERE genres LIKE '%""" + genre + """%'"""
        df = pysqldf(q)
        df['genre'] = genre
        dataframes.append(df)
    return pd.concat(dataframes)

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

def number_in_genre(genres_list):
    dataframes = []
    for genre in genres_list:
        q = """SELECT COUNT(*) 
               FROM movies 
               WHERE genres LIKE '%""" + genre + """%'"""
        df = pysqldf(q)
        df['genre'] = genre
        dataframes.append(df)
    return pd.concat(dataframes)

In [ ]:
all_movie_genres = number_in_genre(genres)

#extracted to numpy array for chi2 test
all_movie_genres_array = all_movie_genres['COUNT(*)'].to_numpy()

all_movie_genres.sort_values(by=["COUNT(*)"], inplace=True)
all_movie_genres

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

all_movie_genres.plot.barh(x='genre', y='COUNT(*)', label='All Movies', ax=ax)
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.title('Number of Movies by Genre', fontsize=15);

In [ ]:
top_30_movies = movies.head(30)

def number_in_genre(genres_list):
    dataframes = []
    for genre in genres_list:
        q = """SELECT COUNT(*) 
               FROM top_30_movies 
               WHERE genres LIKE '%""" + genre + """%'"""
        df = pysqldf(q)
        df['genre'] = genre
        dataframes.append(df)
    return pd.concat(dataframes)

top_movie_genres = number_in_genre(genres)
top_movie_genres_array = top_movie_genres['COUNT(*)'].to_numpy()
top_movie_genres

top_movie_genres.sort_values(by=["COUNT(*)"], inplace=True)
top_movie_genres

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

top_movie_genres.plot.barh(x='genre', y='COUNT(*)', label='Top 30 Movies', ax=ax)
plt.xlabel('Number of Movies')
plt.ylabel('Genre')
plt.title('Number of Movies by Genre', fontsize=15);

In [ ]:
for table in [all_movie_genres, top_movie_genres]:
    table['Percent of'] = table['COUNT(*)'].map(lambda x: x/(table['COUNT(*)'].sum()))
    table.drop('COUNT(*)', axis=1, inplace=True)

In [ ]:
all_movie_genres.rename(columns={'Percent of':'Percent of All Movies'}, inplace=True)
top_movie_genres.rename(columns={'Percent of':'Percent of Top 30 Movies'}, inplace=True)

In [ ]:
genre_comparison = all_movie_genres.merge(top_movie_genres, on='genre')
genre_comparison.sort_values(by=['Percent of Top 30 Movies'], ascending=True, inplace=True)

genre_comparison = genre_comparison.loc[genre_comparison['Percent of Top 30 Movies'] != 0]

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

genre_comparison.plot(x='genre', y=['Percent of All Movies', 'Percent of Top 30 Movies'],
                     kind="barh", ax=ax)
plt.title("Percent of Movies by Genre");

In [ ]:
contingency_table = np.array([all_movie_genres_array,
                             top_movie_genres_array])
contingency_table

In [ ]:
stats.chi2_contingency(contingency_table)

The extremely low p-value here suggests that these two groups are performing differently.